In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import optuna
import tensorflow as tf
from tensorflow import keras

In [2]:
gef = np.loadtxt('GEFCOM.txt')
cols = ['date', 'hour', 'price', 'sysload', 'zoneload', 'dotw']

After loading the data, we create two forecasts - AR(1) with single estimation in two variants: jointly and separately for all hours of the day.

In [4]:
# base cases - AR(1) model modeled jointly for all hours of the day and separately and jointly
X = gef[:, 2:4].copy()
X[:, 1] = 1
X[:, 0] = np.nan
X[24:, 0] = gef[:-24, 2] # the first lag of the time series is inserted into the X matrix, starting at the 24th row
Y = gef[:, 2].copy() # the dependent variable Y is set to the second column of the gef array
firstind = 24
lastind = np.argwhere(gef[:, 0] == 20130101).squeeze()[0] # searching for the first occurrence of the date 20130101 in the first column of the gef array
# squeeze() method is used to convert the resulting one-dimensional array to a scalar
Xt = X[firstind:lastind]
Xf = X[lastind:]
Yt = Y[firstind:lastind]
Yf = Y[lastind:]
betas = np.linalg.lstsq(Xt, Yt, rcond=None)[0]
betas
forecast = np.dot(Xf, betas) # matrix multiplying the test set Xf by the estimated coefficients betas
forecast.shape == Yf.shape
print('MAE of Least-squares jointly: ', np.mean(np.abs(forecast - Yf)))

separate = np.zeros_like(forecast) # all elements initialized to zero, the same shape and data type as forecast
for h in range(24): # model is fit separately for each hour of the day
    betas = np.linalg.lstsq(Xt[h::24], Yt[h::24], rcond=None)[0]
    separate[h::24] = np.dot(Xf[h::24], betas)
print('MAE of Least-squares separately: ', np.mean(np.abs(separate - Yf)))

MAE of Least-squares jointly:  8.37476435765813
MAE of Least-squares separately:  8.259542237368105


The first NN model
------------------

Recreate the joint AR(1) using a Neural Network.

Note, that we are using a simple architecture, with no hidden layers. Rerun the cell below multiple times and see, that the error metric changes each time.

In [5]:
%%time # magic command in Jupyter Notebook that allows you to time how long it takes for a particular cell to run
# NN model for AR(1) jointly for all hours
inputs = keras.Input(2,) # define input layer - 2 independent variables
# hidden = keras.layers.Dense(20, activation='relu')(inputs)
outputs = keras.layers.Dense(1, activation='linear')(inputs)
model = keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())
model.compile(optimizer=keras.optimizers.Adam(), loss='mse', metrics='mae')
# specifies the optimizer to use during training (Adam), the loss function to use - MSE, and the evaluation metric to use - MAE

# set out 20% data for validation
perm = np.random.permutation(np.arange(Xt.shape[0]))
VAL_DATA = 0.2
trainsubset = perm[:int((1 - VAL_DATA) * len(perm))]
valsubset = perm[int((1 - VAL_DATA) * len(perm)):]
model.fit(Xt[trainsubset], Yt[trainsubset], epochs=75, validation_data=(Xt[valsubset], Yt[valsubset]), verbose=True, batch_size=256)
# epochs parameter specifies the number of times to iterate over the entire training dataset during training
# batch_size parameter specifies the number of samples to use for each update of the model weights
NNpred = model.predict(Xf)[:, 0] # This passes the test inputs Xf through the trained model to generate predicted outputs
print(NNpred.shape, Yf.shape, (NNpred - Yf).shape)
print(np.mean(np.abs(NNpred - Yf))) #MAE

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense (Dense)               (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/75
55/55 [==============================] - 1s 7ms/step - loss: 271.7316 - mae: 11.1995 - val_loss: 252.3918 - val_mae: 10.3432
Epoch 2/75
55/55 [==============================] - 0s 2ms/step - loss: 233.3733 - mae: 9.4846 - val_loss: 218.8917 - val_mae: 8.8383
Epoch 3/75
55/55 [==============================] - 0s 2ms/step - loss: 211.2326 - mae: 8.2902 - val_loss: 200.5518 - val_mae: 7.8822
Epoch 4/75
55/55 [==============================] - 0s

Epoch 57/75
55/55 [==============================] - 0s 4ms/step - loss: 181.6408 - mae: 6.3174 - val_loss: 174.6066 - val_mae: 6.3750
Epoch 58/75
55/55 [==============================] - 0s 4ms/step - loss: 181.5670 - mae: 6.3301 - val_loss: 174.6501 - val_mae: 6.3791
Epoch 59/75
55/55 [==============================] - 0s 4ms/step - loss: 181.2557 - mae: 6.3050 - val_loss: 174.3083 - val_mae: 6.3693
Epoch 60/75
55/55 [==============================] - 0s 4ms/step - loss: 181.1541 - mae: 6.3141 - val_loss: 174.1568 - val_mae: 6.3669
Epoch 61/75
55/55 [==============================] - 0s 3ms/step - loss: 180.9240 - mae: 6.3118 - val_loss: 174.2521 - val_mae: 6.3718
Epoch 62/75
55/55 [==============================] - 0s 3ms/step - loss: 180.7231 - mae: 6.3075 - val_loss: 173.9138 - val_mae: 6.3644
Epoch 63/75
55/55 [==============================] - 0s 4ms/step - loss: 180.4962 - mae: 6.3063 - val_loss: 174.1418 - val_mae: 6.3727
Epoch 64/75
55/55 [==============================] - 0s

The second NN model
------------------

Let us add some complexity to the model.

We add two additional layers:
 - a BatchNormalization layer
 - a hidden layer
 
The batch normalization is useful if the input data is not normalized (e.g., we forecast prices using past prices with values ranging from -100 to 300 and load forecasts with values in tens of thousands) - so it is not really helpful here.

Batch normalization normalizes the activations of the previous layer at each batch, i.e., it centers and scales the inputs to have zero mean and unit variance. This helps to prevent the activations from becoming too large or too small during training, which can slow down training or lead to numerical instability.

The model might perform worse than the simpler one - but that just shows that the amount of input data is too low (the NN is able to model highly complex non-linear relations in the data, but the data we pass to the NN is very scarce).

hidden = keras.layers.Dense(5, activation='elu')(bn) creates a dense layer with 5 units and the 'elu' activation function. This layer takes the output from the batch normalization layer and applies a linear transformation to it, followed by the activation function. The 'elu' activation function is the Exponential Linear Unit, which is a variant of the Rectified Linear Unit (ReLU) that allows for negative values. This can help to alleviate the vanishing gradient problem, which can occur when using the ReLU activation function.

In [6]:
# NN model for AR(1) jointly for all hours, improved
inputs = keras.Input(2,) # define input layer - 2 independent variables

bn = keras.layers.BatchNormalization()(inputs)

hidden = keras.layers.Dense(5, activation='elu')(bn)

outputs = keras.layers.Dense(1, activation='linear')(hidden)
model = keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())
model.compile(optimizer=keras.optimizers.Adam(), loss='mse', metrics='mae')
# set out 20% data for validation
perm = np.random.permutation(np.arange(Xt.shape[0]))
VAL_DATA = 0.2
trainsubset = perm[:int((1 - VAL_DATA) * len(perm))]
valsubset = perm[int((1 - VAL_DATA) * len(perm)):]
model.fit(Xt[trainsubset], Yt[trainsubset], epochs=30, validation_data=(Xt[valsubset], Yt[valsubset]), verbose=True, batch_size=128)
NNpred = model.predict(Xf)[:, 0]
print(NNpred.shape, Yf.shape, (NNpred - Yf).shape)
print(np.mean(np.abs(NNpred - Yf)))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2)]               0         
                                                                 
 batch_normalization (BatchN  (None, 2)                8         
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 5)                 15        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 29
Trainable params: 25
Non-trainable params: 4
_________________________________________________________________
None
Epoch 1/30
110/110 [==============================] - 2s 5ms/step - loss: 2633.3181 - mae: 46.0509 - val_loss: 2447.7300 - v

The multi-output model
----------------------

Now, the model will have 24 outputs, one for each hour of the day. At the same time, the number of inputs needs to grow (we need to include the information about all hours of the day).

Here, the scarcity of the model might be even more visible - as we still use only a fraction of the data we have available (and the network can handle).

In [7]:
# NN model for AR(1) with 24 outputs
inputs = keras.Input(25,) # define input layer - 1 independent variable for 24h + ones
bn = keras.layers.BatchNormalization()(inputs) # Apply batch normalization to the input layer
hidden = keras.layers.Dense(50, activation='elu')(bn) # Define a hidden layer with 50 nodes and an ELU activation function
outputs = keras.layers.Dense(24, activation='linear')(hidden) # Define the output layer with 24 nodes, corresponding to the 24 hours of the day
model = keras.Model(inputs=inputs, outputs=outputs) #Define the NN model by specifying the input and output layers
print(model.summary())
model.compile(optimizer=keras.optimizers.Adam(), loss='mse', metrics='mae')
# Compile the NN model using the Adam optimizer, mean squared error (MSE) loss function, and mean absolute error (MAE) metric
# set out 20% data for validation

VAL_DATA = 0.2
Xt24 = np.hstack([Xt[:, 0].reshape(len(Xt) // 24, 24), Xt[::24, 1:]])
perm = np.random.permutation(np.arange(Xt24.shape[0]))
Xf24 = np.hstack([Xf[:, 0].reshape(len(Xf) // 24, 24), Xf[::24, 1:]])
Y24 = Y.reshape(len(Y) // 24, 24)
Yf24 = Yf.reshape(len(Yf) // 24, 24)
trainsubset = perm[:int((1 - VAL_DATA) * len(perm))]
valsubset = perm[int((1 - VAL_DATA) * len(perm)):]
model.fit(Xt24[trainsubset], Y24[trainsubset], epochs=50, validation_data=(Xt24[valsubset], Y24[valsubset]), verbose=True, batch_size=16)
# Train the NN model using the training dataset and the validation dataset for 50 epochs, a batch size of 16, and verbose output
NNpred = model.predict(Xf24)[:, :]
# Use the trained NN model to predict the 24-hour outputs for the test dataset Xf24, and compute the mean absolute error (MAE) between the predicted outputs NNpred and the true outputs Yf24
print(NNpred.shape, Yf24.shape, (NNpred - Yf24).shape)
print(np.mean(np.abs(NNpred - Yf24)))

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 25)]              0         
                                                                 
 batch_normalization_1 (Batc  (None, 25)               100       
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 50)                1300      
                                                                 
 dense_4 (Dense)             (None, 24)                1224      
                                                                 
Total params: 2,624
Trainable params: 2,574
Non-trainable params: 50
_________________________________________________________________
None
Epoch 1/50
37/37 [==============================] - 1s 10ms/step - loss: 2616.6648 - mae: 46.1386 - val_loss: 2228.75

Hyper-parameter optimization
----------------------------

In the previous examples, the parameters such as number of neurons were set at some level. However, the neural networks have a lot of hyper-parameters (especially the larger ones), for example, we can even have a hyper-parameter that decides whether to include a data processing step or to include a given input variable in the model.

Hyper-parameters are, by definition, parameters of the model that are not "trained" when training the model (the "trainable parameters" are the weights in the network, correponding to the beta coefficients in linear regression). We need to set the hyper-parameters prior to training the neural network, their values are typically determined in the separate optimization study.

Here, we will optimize the number of neurons using optuna package. Note, that we need to set out some data to evaluate the hyper-parameter sets (and this set should be separate from the out-of-sample test that will be used to evaluate the model after choosing the hyper-parameters).

In [8]:
import optuna
import logging
import sys
Xt24 = np.hstack([Xt[:, 0].reshape(len(Xt) // 24, 24), Xt[::24, 1:]])
Xf24 = np.hstack([Xf[:, 0].reshape(len(Xf) // 24, 24), Xf[::24, 1:]])
Y24 = Y.reshape(len(Y) // 24, 24)
Y24 = Y24[:len(Xt24)]
Yf24 = Yf.reshape(len(Yf) // 24, 24)

# hyper-parameter optimization step
# we will use the part of the training data - with the last 60 days being left out for the evaluation
Xt_opti = Xt24[:-60]
Xf_opti = Xt24[-60:]
Yt_opti = Y24[:-60]
Yf_opti = Y24[-60:]
print(Xt_opti.shape, Xf_opti.shape, Yt_opti.shape, Yf_opti.shape)

def objective(trial):
    # function for the hyper-parameter optimization of the neural network model. The function takes in a trial object from the Optuna 
    #library, which contains the parameters being optimized. The function defines a neural network with one hidden layer and the number 
    #of hidden neurons and the activation function of the hidden layer are optimized. The function returns the mean absolute error between the predicted and actual values on the validation set.
    
    # Build a NN that has one hidden layer and optimize the number of hiden neurons and the hidden activation
    neurons = trial.suggest_int('neurons', 3, 100)
    activation = trial.suggest_categorical('activation', ['elu', 'relu', 'linear', 'sigmoid'])
    # the rest of the parameters will be fixed
    inputs = keras.Input(25,) # define input layer - 1 independent variable for 24h + ones
    bn = keras.layers.BatchNormalization()(inputs)
    hidden = keras.layers.Dense(neurons, activation=activation)(bn)
    outputs = keras.layers.Dense(24, activation='linear')(hidden)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=keras.optimizers.Adam(), loss='mse', metrics='mae')
    # note we don't have the validation here - this is only to simplify, it can and should be used
    model.fit(Xt_opti, Yt_opti, epochs=50, verbose=False, batch_size=16)
    NNpred = model.predict(Xf_opti)[:, :]
    return np.mean(np.abs(NNpred - Yf_opti))

optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout)) # sets up logging to output the results of the optimization to the console
study_name = 'study'
storage_name = 'sqlite:///study.sqlite'

study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=False) # creates a new study object with the specified name and database storage location
study.optimize(objective, n_trials=100, show_progress_bar=True) # runs the optimization process on the defined objective function, with 100 trials to search the hyperparameter space and a progress bar to display the status of the optimization

best_params = study.best_params #  retrieves the best set of hyperparameters found by the optimization process
print(best_params)

# NN model for AR(1) with 24 outputs with hyper-parameter optimization -- using the optimized parameters
inputs = keras.Input(25,) # define input layer - 1 independent variable for 24h + ones
bn = keras.layers.BatchNormalization()(inputs)
hidden = keras.layers.Dense(best_params['neurons'], activation=best_params['activation'])(bn)
outputs = keras.layers.Dense(24, activation='linear')(hidden)
model = keras.Model(inputs=inputs, outputs=outputs)
print(model.summary())
model.compile(optimizer=keras.optimizers.Adam(), loss='mse', metrics='mae')
# set out 20% data for validation

VAL_DATA = 0.2
perm = np.random.permutation(np.arange(Xt24.shape[0]))

trainsubset = perm[:int((1 - VAL_DATA) * len(perm))]
valsubset = perm[int((1 - VAL_DATA) * len(perm)):]
model.fit(Xt24[trainsubset], Y24[trainsubset], epochs=50, validation_data=(Xt24[valsubset], Y24[valsubset]), verbose=True, batch_size=16)
NNpred = model.predict(Xf24)[:, :]
print(NNpred.shape, Yf24.shape, (NNpred - Yf24).shape)
print(np.mean(np.abs(NNpred - Yf24)))

(670, 25) (60, 25) (670, 24) (60, 24)


[I 2023-05-10 16:20:59,054] A new study created in RDB with name: study


A new study created in RDB with name: study


C:\Users\asia-\anaconda3\lib\site-packages\optuna\progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/100 [00:00<?, ?it/s]

2/2 [==============================] - 0s 3ms/step
Trial 0 finished with value: 3.1743979552586876 and parameters: {'neurons': 51, 'activation': 'relu'}. Best is trial 0 with value: 3.1743979552586876.
[I 2023-05-10 16:21:04,967] Trial 0 finished with value: 3.1743979552586876 and parameters: {'neurons': 51, 'activation': 'relu'}. Best is trial 0 with value: 3.1743979552586876.
2/2 [==============================] - 0s 4ms/step
Trial 1 finished with value: 5.852950577947829 and parameters: {'neurons': 12, 'activation': 'linear'}. Best is trial 0 with value: 3.1743979552586876.
[I 2023-05-10 16:21:10,878] Trial 1 finished with value: 5.852950577947829 and parameters: {'neurons': 12, 'activation': 'linear'}. Best is trial 0 with value: 3.1743979552586876.
2/2 [==============================] - 0s 2ms/step
Trial 2 finished with value: 3.2065070124732125 and parameters: {'neurons': 92, 'activation': 'relu'}. Best is trial 0 with value: 3.1743979552586876.
[I 2023-05-10 16:21:15,316] Trial 

[I 2023-05-10 16:22:29,505] Trial 17 finished with value: 3.330535571681129 and parameters: {'neurons': 78, 'activation': 'linear'}. Best is trial 14 with value: 2.965909490426381.
2/2 [==============================] - 0s 2ms/step
Trial 18 finished with value: 5.359215482976702 and parameters: {'neurons': 58, 'activation': 'sigmoid'}. Best is trial 14 with value: 2.965909490426381.
[I 2023-05-10 16:22:34,859] Trial 18 finished with value: 5.359215482976702 and parameters: {'neurons': 58, 'activation': 'sigmoid'}. Best is trial 14 with value: 2.965909490426381.
2/2 [==============================] - 0s 4ms/step
Trial 19 finished with value: 2.8979457795884875 and parameters: {'neurons': 100, 'activation': 'relu'}. Best is trial 19 with value: 2.8979457795884875.
[I 2023-05-10 16:22:40,272] Trial 19 finished with value: 2.8979457795884875 and parameters: {'neurons': 100, 'activation': 'relu'}. Best is trial 19 with value: 2.8979457795884875.
2/2 [==============================] - 0s 2ms

2/2 [==============================] - 0s 4ms/step
Trial 59 finished with value: 2.8995414358245 and parameters: {'neurons': 92, 'activation': 'relu'}. Best is trial 20 with value: 2.7048438454733956.
[I 2023-05-10 16:25:47,507] Trial 59 finished with value: 2.8995414358245 and parameters: {'neurons': 92, 'activation': 'relu'}. Best is trial 20 with value: 2.7048438454733956.
2/2 [==============================] - 0s 2ms/step
Trial 60 finished with value: 3.16297946744495 and parameters: {'neurons': 98, 'activation': 'linear'}. Best is trial 20 with value: 2.7048438454733956.
[I 2023-05-10 16:25:52,923] Trial 60 finished with value: 3.16297946744495 and parameters: {'neurons': 98, 'activation': 'linear'}. Best is trial 20 with value: 2.7048438454733956.
2/2 [==============================] - 0s 4ms/step
Trial 61 finished with value: 3.604674173037211 and parameters: {'neurons': 93, 'activation': 'relu'}. Best is trial 20 with value: 2.7048438454733956.
[I 2023-05-10 16:25:57,266] Trial

2/2 [==============================] - 0s 3ms/step
Trial 80 finished with value: 5.395171118948195 and parameters: {'neurons': 95, 'activation': 'sigmoid'}. Best is trial 20 with value: 2.7048438454733956.
[I 2023-05-10 16:27:24,536] Trial 80 finished with value: 5.395171118948195 and parameters: {'neurons': 95, 'activation': 'sigmoid'}. Best is trial 20 with value: 2.7048438454733956.
2/2 [==============================] - 0s 3ms/step
Trial 81 finished with value: 3.090227739599016 and parameters: {'neurons': 98, 'activation': 'relu'}. Best is trial 20 with value: 2.7048438454733956.
[I 2023-05-10 16:27:28,694] Trial 81 finished with value: 3.090227739599016 and parameters: {'neurons': 98, 'activation': 'relu'}. Best is trial 20 with value: 2.7048438454733956.
2/2 [==============================] - 0s 4ms/step
Trial 82 finished with value: 2.873054886341095 and parameters: {'neurons': 94, 'activation': 'relu'}. Best is trial 20 with value: 2.7048438454733956.
[I 2023-05-10 16:27:32,23

 input_104 (InputLayer)      [(None, 25)]              0         
                                                                 
 batch_normalization_102 (Ba  (None, 25)               100       
 tchNormalization)                                               
                                                                 
 dense_205 (Dense)           (None, 100)               2600      
                                                                 
 dense_206 (Dense)           (None, 24)                2424      
                                                                 
Total params: 5,124
Trainable params: 5,074
Non-trainable params: 50
_________________________________________________________________
None
Epoch 1/50
37/37 [==============================] - 2s 10ms/step - loss: 2620.8069 - mae: 45.5228 - val_loss: 1411.1556 - val_mae: 34.5011
Epoch 2/50
37/37 [==============================] - 0s 5ms/step - loss: 2259.1536 - mae: 42.0079 - val_loss: 686.6703 - val_mae

In [9]:
print(np.mean(np.abs(NNpred - Yf24)))

9.347363136831286


In [10]:
best_params

{'activation': 'relu', 'neurons': 100}

In [11]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_activation,params_neurons,state
0,0,3.174398,2023-05-10 16:20:59.149699,2023-05-10 16:21:04.908337,0 days 00:00:05.758638,relu,51,COMPLETE
1,1,5.852951,2023-05-10 16:21:05.009577,2023-05-10 16:21:10.817795,0 days 00:00:05.808218,linear,12,COMPLETE
2,2,3.206507,2023-05-10 16:21:10.943480,2023-05-10 16:21:15.280943,0 days 00:00:04.337463,relu,92,COMPLETE
3,3,3.118102,2023-05-10 16:21:15.357263,2023-05-10 16:21:22.269394,0 days 00:00:06.912131,relu,35,COMPLETE
4,4,4.917021,2023-05-10 16:21:22.369381,2023-05-10 16:21:28.245127,0 days 00:00:05.875746,relu,8,COMPLETE
...,...,...,...,...,...,...,...,...
95,95,2.853811,2023-05-10 16:28:33.177632,2023-05-10 16:28:38.622553,0 days 00:00:05.444921,relu,82,COMPLETE
96,96,3.012801,2023-05-10 16:28:38.739638,2023-05-10 16:28:44.558821,0 days 00:00:05.819183,relu,57,COMPLETE
97,97,3.031410,2023-05-10 16:28:44.646746,2023-05-10 16:28:49.979856,0 days 00:00:05.333110,linear,87,COMPLETE
98,98,3.005826,2023-05-10 16:28:50.066853,2023-05-10 16:28:55.117593,0 days 00:00:05.050740,relu,94,COMPLETE
